In [1]:
import pandas as pd 
import numpy as np
import os  

In [2]:

# Process all the data to get a general format for the graduates people

# Global Variables
general_path  = os.path.abspath(os.path.join(os.getcwd()))
path_preprocessing_data = os.path.abspath(os.path.join(general_path,"Preprocessing_Data"))

# DataFrames

df_preprocessing = pd.DataFrame()


In [3]:
for file in os.listdir(path_preprocessing_data):
    temp_json_file = pd.read_json(os.path.abspath(os.path.join(path_preprocessing_data,file)))

    print(temp_json_file.columns)

Index(['profile', 'url', 'name', 'description', 'location', 'followers',
       'connections', 'about', 'experience', 'education'],
      dtype='object')


In [42]:
temp_json_file['experience'][0]

[{'organisation_profile': 'https://www.linkedin.com/company/turingbox',
  'location': '',
  'description': '',
  'start_time': 'Aug 2022',
  'end_time': 'Apr 2022',
  'duration': None},
 {'organisation_profile': 'https://www.linkedin.com/company/universidadnacionaldecolombia',
  'location': 'Bogotá, Distrito Capital, Colombia',
  'description': 'Se desarrolló un análisis de datos de las tesis de posgrado de la Facultad de ingeniería.',
  'start_time': 'May 2022',
  'end_time': 'Sep 2022',
  'duration': None},
 {'organisation_profile': 'https://www.linkedin.com/company/out-xvii',
  'location': '',
  'description': ''}]

In [6]:
temp_json_file['about'][0]#[0]#['about']#.keys()#.unique()#.info()

In [2]:
# Profile Name -> name (string)
# Profile URL -> url (string)
# Description -> description (string)
# Location (house) -> location (string)
# Education -> (dict) -> organisation_name, organisation_profile

cont = 68

In [5]:
import requests
import json

url = "https://api.prospeo.io/linkedin-email-finder"
api_key = "c591c3d5993ae82882c69ebce9f32f69"

required_headers = {
    'Content-Type': 'application/json',
    'X-KEY': api_key
}

linkedin_profiles_extended = [
    "https://co.linkedin.com/in/seradiazpin",
    "https://co.linkedin.com/in/andrea-millan-75a63b1a2",
    "https://co.linkedin.com/in/nelson-david-navarro-diaz-259741a5",
    "https://co.linkedin.com/in/federico-puentes-acosta-b86388271",
    "https://co.linkedin.com/in/cindy-ramirez-restrepo",
    "https://co.linkedin.com/in/luis-felipe-epia-realpe-b7770a133",
    "https://co.linkedin.com/in/elvira-forero-a191063a?original_referer=https%3A%2F%2Fwww.google.com%2F",
    "https://co.linkedin.com/in/lorena-ram%C3%ADrez-%C3%A1vila-957a2b129",
    "https://co.linkedin.com/in/jeison-julian-lozano-jojoa-57b647114",
    "https://co.linkedin.com/in/santiago-parra-7b6543219"

]


json_aggregate = []


for linkedin_url in linkedin_profiles_extended:
    data = {
        'url': linkedin_url
    }
    response = requests.post(url, json=data, headers=required_headers)
    
    if response.status_code == 200:
        print(f"Processing {linkedin_url}")
        json_content = response.json()
        json_aggregate.append({linkedin_url: json_content})
        
        # Save each JSON data to a uniquely named file
        filename = f"./Preprocessing_Data/data_json/data_file_{cont}.json"  # The filename contains the counter variable
        with open(filename, 'w') as f:
            json.dump({linkedin_url: json_content}, f, separators=(',', ':'))
        
        cont += 1  # Increment your counter
    else:
        print(f"Error for {linkedin_url}: Unable to fetch data. HTTP Status Code: {response.status_code}")

# Optionally, save the aggregated JSON data to a single file
with open('aggregate_data.json', 'w') as f:
    json.dump(json_aggregate, f, separators=(',', ':'))


Processing https://co.linkedin.com/in/alejandro-marin-cardona?challengeId=AQE3ida8k1R3_gAAAYs5WzzJrk_Iu92lql9CnT-Uc14AaXAXQXm8EPs1MQ908aJg__Y9BIuqOVhPMe3zjDBXkZMr3bevpfDWtw&submissionId=99e06f56-f0a3-8e17-ef7a-1053cd889601&challengeSource=AgGnSPQjmsltKQAAAYs5W7uFeI2AXgxqcdJbjWKP6snWAYKpaGUVQW01npkdgV4&challegeType=AgG77SxP0U5rawAAAYs5W7uISh4BqRE1NUYt0b7wKcNJqwlqXNXaWcw&memberId=AgHrPQhxE6KR3QAAAYs5W7uMQ7AMpPozsGO5_v_TAtYYkQw&recognizeDevice=AgGdmPHAOcy5qwAAAYs5W7uPQx0jYYOFMo2f6Gpvx2WGoFIwTlzo
Processing https://www.linkedin.com/in/daniel-mora14
Processing https://co.linkedin.com/in/johnattan-fernando-devia-rojas-8b42276a
Processing https://co.linkedin.com/in/christopher-rodr%C3%ADguez-fonseca-270464158
Processing https://co.linkedin.com/in/maikol1804
Processing https://www.linkedin.com/in/amrondonp
Processing https://co.linkedin.com/in/john-jairo-camargo-bolivar-31a899136
Processing https://co.linkedin.com/in/alegonzalezc
Processing https://co.linkedin.com/in/cristian-andres-melo-lopez